In [1]:
import wandb
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

In [2]:
from jeremy_ft import get_pets

In [4]:
PROJECT  = "fine_tune_timm"
GROUP    = "timm"
JOB_TYPE = "downstream_task"

MODEL_NAME = "regnetx_040"

In [5]:
config = SimpleNamespace(
    batch_size=64,
    img_size=224,
    seed=42,
    pretrained=False,
    model_name=MODEL_NAME,
    dataset="PETS",
    epochs=5)

## Train from scratch

In [6]:
def train(config):
    dls = get_pets(config.batch_size, config.img_size, config.seed)
    with wandb.init(project=PROJECT, group=GROUP, job_type=JOB_TYPE, config=config):
        cbs = [MixedPrecision(), WandbCallback(log_preds=False)]
        learn = vision_learner(dls, config.model_name, metrics=[accuracy, error_rate], 
                               cbs=cbs, pretrained=config.pretrained)
        learn.fine_tune(config.epochs)

In [7]:
for _ in range(5):
    train(config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape (use `wandb login --relogin` to force relogin)


WandbCallback requires use of "SaveModelCallback" to log best model


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,4.472122,3.600423,0.100812,0.899188,00:23


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,3.914697,3.306748,0.124493,0.875507,00:18
1,3.774834,3.310742,0.129229,0.870771,00:19
2,3.599136,3.221924,0.143437,0.856563,00:19
3,3.362622,3.142187,0.161705,0.838295,00:19
4,3.261196,3.135579,0.166441,0.833559,00:19


accuracy,▁▄▄▆▇█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
error_rate,█▅▅▃▂▁
lr_0,▁▂▃▅▆▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
lr_2,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
mom_0,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████
mom_1,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████


...

accuracy,▁▃▄▅▇█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
error_rate,█▆▅▄▂▁
lr_0,▁▂▃▅▆▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
lr_2,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
mom_0,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████
mom_1,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████


WandbCallback requires use of "SaveModelCallback" to log best model


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,4.484125,3.593723,0.109608,0.890392,00:19


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,3.850176,3.296687,0.119080,0.880920,00:19
1,3.789353,3.278631,0.131935,0.868065,00:19
2,3.596220,3.211500,0.136671,0.863329,00:19
3,3.405158,3.153484,0.147497,0.852503,00:19
4,3.270511,3.135779,0.156969,0.843031,00:18


accuracy,▁▂▄▅▇█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
error_rate,█▇▅▄▂▁
lr_0,▁▂▃▅▆▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
lr_2,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
mom_0,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████
mom_1,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████


## Pretrained

In [8]:
config.pretrained = True
for _ in range(5):
    train(config)

WandbCallback requires use of "SaveModelCallback" to log best model


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.109841,0.272206,0.904601,0.095399,00:20


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.326107,0.201115,0.939784,0.060217,00:20
1,0.280189,0.242307,0.919486,0.080514,00:20
2,0.201097,0.203991,0.930988,0.069012,00:20
3,0.145418,0.181045,0.935047,0.064953,00:20
4,0.104143,0.174602,0.945196,0.054804,00:20


accuracy,▁▇▄▆▆█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
error_rate,█▂▅▃▃▁
lr_0,▁▂▃▅▆▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
lr_2,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
mom_0,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████
mom_1,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████


WandbCallback requires use of "SaveModelCallback" to log best model


...

accuracy,▁▁▃▆██
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
error_rate,██▆▃▁▁
lr_0,▁▂▃▅▆▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
lr_2,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
mom_0,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████
mom_1,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████


WandbCallback requires use of "SaveModelCallback" to log best model


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.100217,0.281893,0.903924,0.096076,00:20


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.332728,0.220499,0.927605,0.072395,00:20
1,0.288674,0.235575,0.926928,0.073072,00:20
2,0.221376,0.190746,0.937077,0.062923,00:20
3,0.145828,0.173737,0.943843,0.056157,00:20
4,0.112507,0.174783,0.941813,0.058187,00:20


accuracy,▁▅▅▇██
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
error_rate,█▄▄▂▁▁
lr_0,▁▂▃▅▆▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
lr_2,▁▂▃▅▆▇█▂▂▂▃▃▃▄▄▄▄▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
mom_0,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████
mom_1,██▆▄▃▂▁██▇▆▅▄▃▂▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇▇████


## Planets

In [10]:
planets_config = SimpleNamespace(
    batch_size=64,
    img_size=224,
    seed=42,
    pretrained=True,
    model_name=MODEL_NAME,
    dataset="PLANETS",
    epochs=5)

In [11]:
def get_planets(batch_size=64, img_size=224, seed=42):
    dataset_path=untar_data(URLs.PLANET_SAMPLE)
    dls = ImageDataLoaders.from_csv(dataset_path, 
                                    folder="train", 
                                    csv_fname="labels.csv",
                                    label_delim=" ",
                                    suff=".jpg",
                                    bs=batch_size,
                                    seed=seed,
                                    item_tfms=Resize(img_size))
    return dls

## Log dataset

In [ ]:
dls = get_planets()

In [ ]:
dls.vocab

In [ ]:
dls.train_ds

In [85]:
data = []
for i, (img, labels) in enumerate(dls.valid_ds):
    if i>36: break
    data.append([wandb.Image(img)] + list(map(int, labels)))

In [91]:
table = wandb.Table(data=data, columns = ["image"]+list(dls.vocab))

In [93]:
with wandb.init(project=PROJECT, group=GROUP, job_type="log_planets"):
    wandb.log({"planets_sample": table})

## Train

In [16]:
def train_planets(config):
    dls = get_planets(config.batch_size, config.img_size, config.seed)
    with wandb.init(project=PROJECT, group=GROUP, job_type=JOB_TYPE, config=config):
        cbs = [MixedPrecision(), WandbCallback(log_preds=False)]
        learn = vision_learner(dls, config.model_name, metrics=[accuracy_multi,F1ScoreMulti()], 
                               cbs=cbs, pretrained=config.pretrained)
        learn.fine_tune(config.epochs)

## Scratch

In [ ]:
planets_config.pretrained = False
for _ in range(5):
    train_planets(planets_config)

WandbCallback requires use of "SaveModelCallback" to log best model


epoch,train_loss,valid_loss,accuracy_multi,f1_score,time
0,0.948215,0.706917,0.638824,0.070742,00:03


epoch,train_loss,valid_loss,accuracy_multi,f1_score,time
0,0.805229,0.579893,0.874412,0.105127,00:03
1,0.793956,0.572235,0.791471,0.142315,00:03
2,0.766795,0.494692,0.809412,0.248048,00:03
3,0.738460,0.504847,0.800000,0.341506,00:03
4,0.718422,0.523365,0.802353,0.348073,00:03


## Pretrained

In [ ]:
planets_config.pretrained = True
for _ in range(5):
    train_planets(planets_config)